# Experimento 1
***
- Conjunto de Dados: NIH
- Aplicando mudança de escala típica nas imagens

### Importação dos pacotes

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

### Importação dos dados
***
- Todo o dataset foi pré-processado anteriormente

In [2]:
# importando os dataframes dos dados de treinamento, validação e teste
train_df = pd.read_csv('/content/drive/MyDrive/train_df.csv', sep = ',', index_col=  0)
validation_df = pd.read_csv('/content/drive/MyDrive/validation_df.csv', sep = ',', index_col=  0)
test_df = pd.read_csv('/content/drive/MyDrive/test_df.csv' , sep = ',', index_col=  0)

In [3]:
# tornando as classes na coluna 'labels' categórica
train_df.loc[train_df.labels == 1, 'labels'] = 'abnormal'
train_df.loc[train_df.labels == 0, 'labels'] = 'normal'

validation_df.loc[validation_df.labels == 1, 'labels'] = 'abnormal'
validation_df.loc[validation_df.labels == 0, 'labels'] = 'normal'

test_df.loc[test_df.labels == 1, 'labels'] = 'abnormal'
test_df.loc[test_df.labels == 0, 'labels'] = 'normal'

In [4]:
# ajustando a quantidade de exemplos nos dados de treinamento relativo a radiográficas normais
train_normal_cases = train_df[train_df.labels == 'normal']
train_abnormal_cases = train_df[train_df.labels == 'abnormal']
train_normal_cases, _ = train_test_split(train_normal_cases, test_size = 0.5, random_state = 42)
train = np.concatenate((train_normal_cases, train_abnormal_cases))
train_df = pd.DataFrame(train, columns = ['Image Index', 'finding_labels', 'labels'])

In [5]:
# visualizando como é a organização do dataframe pré-processado
train_df.head()

,Image Index,finding_labels,labels
0,/content/drive/MyDrive/images-nih4/images/0002...,No Finding,normal
1,/content/drive/MyDrive/images-nih3/images/0001...,No Finding,normal
2,/content/drive/MyDrive/images-nih2/images/0000...,No Finding,normal
3,/content/drive/MyDrive/images-nih2/images/0000...,No Finding,normal
4,/content/drive/MyDrive/images-nih2/images/0001...,No Finding,normal


In [6]:
# visualizando a quantidade de imagens disponíveis por classe
train_df['labels'].value_counts()

normal      27811
abnormal    25452
Name: labels, dtype: int64

In [7]:
# organizando um dicionário para realizar o balanceamento nos dados das classes
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_df['labels']),
                                                  train_df['labels'])
class_weight = {0: class_weights[0], 1: class_weights[1]}

In [8]:
# normalizando as imagens de treinamento e aplicando aumento de dados
image_generator = ImageDataGenerator(rescale = 1./255., rotation_range = 10, zoom_range = 0.2)

# criando o gerador de imagens de treinamento 
train_generator = image_generator.flow_from_dataframe(
                                                      dataframe = train_df,
                                                      x_col = 'Image Index',
                                                      y_col = 'labels',
                                                      batch_size = 32,
                                                      seed = 42,
                                                      shuffle = True,
                                                      class_mode = 'categorical',
                                                      color_mode = 'rgb',
                                                      target_size = (256, 256))

# normalizando as imagens de teste 
test_datagen = ImageDataGenerator(rescale = 1./255.)

# criando o gerador de imagens de validação 
valid_generator = test_datagen.flow_from_dataframe(
                                                    dataframe = validation_df,
                                                    x_col = 'Image Index',
                                                    y_col = 'labels',
                                                    batch_size = 32,
                                                    seed = 42,
                                                    shuffle = True,
                                                    class_mode = 'categorical',
                                                    target_size = (256, 256))

test_generator = test_datagen.flow_from_dataframe(
                                                  dataframe = test_df, 
                                                  x_col = 'Image Index',
                                                  y_col = 'labels',
                                                  batch_size = 32,
                                                  seed = 42,
                                                  shuffle = True,
                                                  class_mode = 'categorical',
                                                  target_size = (256, 256))

Found 53263 validated image filenames belonging to 2 classes.
Found 4268 validated image filenames belonging to 2 classes.
Found 2640 validated image filenames belonging to 2 classes.


In [9]:
# visualizando a ordem numérica das classes nos dados de treinamento
train_generator.class_indices

{'abnormal': 0, 'normal': 1}

### Preparando a rede neural convolucional 

In [10]:
# baixando os pesos treinados da rede inception
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-04-20 13:35:52--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 74.125.20.128, 74.125.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   124MB/s    in 0.7s    

2021-04-20 13:35:53 (124 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [11]:
# referenciando o local em que os pesos estão armazenados
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# carregando a arquitetura inception pré-treinada
pre_trained_model = InceptionV3(input_shape = (256, 256, 3), include_top = False, weights = None)

# carregando os pesos treinados com outros dados 
pre_trained_model.load_weights(local_weights_file)

# definindo as flags iniciais  
pre_trained_model.trainable = True
set_trainable = False

# para a arquitetura inception, a rede será retreinada a partir da camada 'mixed6'
for layer in pre_trained_model.layers:
    if layer.name == 'mixed6':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

# obtendo a última camada como sendo a nomeada por 'mixed7'
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [12]:
# definindo uma camada de achatamento
x = layers.GlobalAveragePooling2D()(last_output)
# adicionando uma camada densa com 512 neurônios
x = layers.Dense(units = 512, activation = tf.nn.relu)(x)     
# conecatando a rede uma camada com 128 neurônios e função de ativação relu
x = layers.Dense(units = 256, activation = tf.nn.relu)(x) 
# aplicando uma camada de dropout com uma taxa de 20% (normalização)
x = layers.Dropout(rate = 0.2)(x)             
# adicionando uma camada de saída com um neurônio e uma função de ativação sigmoide
x = layers.Dense  (units = 2, activation = tf.nn.softmax)(x)           

# conecatando as camadas definidas acima com a arquitetura inception
model = Model(pre_trained_model.input, x) 

# compilando a rede 
model.compile(optimizer = optimizers.RMSprop(learning_rate = 0.001), loss = 'categorical_crossentropy', 
              metrics = ['acc']) 

In [13]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "transferlearning_weights.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')

In [14]:
# definindo um array de callbacks
callbacks = [checkpoint]

In [ ]:
# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 53259 // 32, 
                              validation_data = valid_generator, validation_steps = 4268 // 32,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight,
                              use_multiprocessing = True, workers = 8)

Epoch 1/10
208/208 [==============================] - 4185s 19s/step - loss: 0.6623 - acc: 0.6230 - val_loss: 0.6516 - val_acc: 0.6700

Epoch 00001: val_acc improved from -inf to 0.66998, saving model to transferlearning_weights.hdf5
Epoch 2/10
208/208 [==============================] - 638s 3s/step - loss: 0.6303 - acc: 0.6563 - val_loss: 0.6787 - val_acc: 0.6297

Epoch 00002: val_acc did not improve from 0.66998
Epoch 3/10
208/208 [==============================] - 642s 3s/step - loss: 0.6189 - acc: 0.6650 - val_loss: 0.6219 - val_acc: 0.6626

Epoch 00003: val_acc did not improve from 0.66998
Epoch 4/10
175/208 [========================>.....] - ETA: 1:34 - loss: 0.6166 - acc: 0.6695

In [ ]:
# avaliando a capacidade de predição do modelo com os dados de teste
model.evaluate(test_generator)

In [ ]:
# salvando o modelo treinado
model.save('/content/drive/MyDrive/weights-nih/inception/model_binv1.0')

### Continuação do Treinamento

In [ ]:
# carregando o modelo salvo
model = tf.keras.models.load_model('/content/drive/MyDrive/weights-nih/inception/model_binv1.0')
# carregando os pesos treinados com o modelo
model.load_weights('/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.1.hdf5')

In [ ]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.1.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
# definindo um array de callbacks
callbacks = [checkpoint]

# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 53263 // 256, 
                              validation_data = valid_generator, validation_steps = 4268 // 128,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight,
                              use_multiprocessing = True, workers = 8)

In [ ]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.2.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
# definindo um array de callbacks
callbacks = [checkpoint]

# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 53263 // 256, 
                              validation_data = valid_generator, validation_steps = 4268 // 128,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight,
                              use_multiprocessing = True, workers = 8)

In [ ]:
# salvando o modelo treinado
model.save('/content/drive/MyDrive/weights-nih/inception/model_binv1.2')

In [ ]:
# testando a capacidade de predição do modelo com os dados de teste
model.evaluate(test_generator)

### Continuação da etapa de treinamento

In [ ]:
# carregando os pesos previamente treinados
model.load_weights('/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.2.hdf5')

In [ ]:
# definindo o caminho pelo qual os pesos serão armazenados 
filepath = "/content/drive/MyDrive/weights-nih/inception/transferlearning_weights_bin_v1.3.hdf5"
# callback para salvar o melhor valor dos pesos em relação ao desempenho com os dados de validação 
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
# definindo um array de callbacks
callbacks = [checkpoint]

# treinando a rede neural convolucional
history = model.fit_generator(train_generator, steps_per_epoch = 53263 // 256, 
                              validation_data = valid_generator, validation_steps = 4268 // 128,
                              callbacks = callbacks, epochs = 10, class_weight = class_weight,
                              use_multiprocessing = True, workers = 8)

In [ ]:
# salvando o modelo treinado
model.save('/content/drive/MyDrive/weights-nih/inception/model_binv1.3')